In [1]:
### Filter and normalize data from E1 to prepare for MOFA input

#############################################
# Prerequisites - Load Libraries

In [2]:
source('MS0_Libraries.r')

In [14]:
source('MS4_Plot_Config.r')

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


###############################################
# Preqrequisites Configurations & Parameters

In [15]:
data_path =   '../data/current'

In [16]:
result_path =  '../results/current'

In [17]:
standardize = FALSE

In [18]:
set_zero_na = FALSE

In [19]:
quantile_norm_feat = TRUE

In [20]:
#samples_var = 'acs_ccs'
# samples_var = 'no_acs'
samples_var = 'all'
# samples_var = 'acs'

In [21]:
name = 'V_FINAL'   # name of dataset to load
name_save = 'V_FINAL_INTEGRATED'


# Functions

In [22]:
### Function for quantile normalization

quantile_normalization = function(X){
  set.seed(42)
  ranks = apply(X, 2, rank, ties.method = 'min')  # determine ranks of each entry
  
  sorted = data.frame(apply(X, 2, sort)) # sort the entries
  means = apply(sorted, 1, mean) # calculate the means
  
  normalized_data = apply(ranks, 2 ,function(x){ means[x]}) # substitute the means into ranks matrix
}


In [23]:
### Gene wise quantile normalization

stdnorm <- function(x) {
  set.seed(42)
  r = rank(x[!is.na(x)], ties.method="average")
  x[!is.na(x)] = qnorm(r / (length(x[!is.na(x)]) + 1))
  return(x)
}

# Load Data 

## Prepared combined data

In [24]:
path = paste0(result_path, '/E-Analysis/Combined_Data_',name,'.csv')
data_long = read.csv(path)
print(file.info(path)$mtime)

[1] "2023-07-18 09:12:52 UTC"


In [25]:
head(data_long,2)

,X,sample_id,type,variable,value,config
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1,k10,single_cell,X0_T.cell.CD4__SSU72,0.5896835,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-
2,2,k10,single_cell,X0_T.cell.CD4__PARK7,0.8587511,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-


In [26]:
nrow(data_long)

[1] 1557939

In [27]:
nrow(unique(data_long[,c('sample_id', 'type', 'variable')]))

[1] 1557939

In [28]:
length(unique(data_long$sample_id))

[1] 146

## Sample Data

In [31]:
sample_data = read.csv(paste0(result_path, '/00_Data_Overview/Merged_Sample_Meta_Data.csv'))

In [33]:
sample_data$CK_raw = sample_data$CK

In [34]:
sample_data$CK = log2(sample_data$CK+1)

In [35]:
sample_data$Troponin = log2(sample_data$Troponin+1)

In [36]:
sample_data$CK_MB = log2(as.numeric(sample_data$CK_MB)+1)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [37]:
sample_data$CRP =  log2(as.numeric(sample_data$CRP)+1)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [38]:
sample_data$sample_id[(sample_data$sample_id == 'm13.2')  & (sample_data$library %in%  c('L6'))] = 'm13.22'

In [39]:
sample_data$sample_id[(sample_data$sample_id == 'm6.4')  & (sample_data$library %in%  c('L10'))] = 'm6.42'

In [40]:
sample_data$sample_id[(sample_data$sample_id == 'm6.1')  & (sample_data$library %in%  c('L3'))] = 'm6.12'

# Potential Pre-Processing / Data Adjustments

###  Filter samples (remove no-acs patients)

In [41]:
head(data_long,2)

,X,sample_id,type,variable,value,config
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1,k10,single_cell,X0_T.cell.CD4__SSU72,0.5896835,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-
2,2,k10,single_cell,X0_T.cell.CD4__PARK7,0.8587511,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-


In [43]:
unique(sample_data$classification)

[1] "vollstaendiger_ausschluss" "ccs"                      
[3] "koronarsklerose"           "acs_subacute"             
[5] "acs_w_infection"           "acs_w_o_infection"

In [44]:
samples_var

[1] "all"

In [45]:
if(samples_var == 'all'){
    samples = unique(sample_data$sample_id) # without filter 
    }
if(samples_var == 'acs_ccs'){
    samples = unique(sample_data$sample_id[!sample_data$classification %in% c('vollstaendiger_ausschluss', 'koronarsklerose')])  # only acs comparison
    }
if(samples_var == 'no_acs'){
    samples = unique(sample_data$sample_id[sample_data$classification %in% c('vollstaendiger_ausschluss', 'ccs', 'koronarsklerose')])  # only acs comparison
    }
if(samples_var == 'acs'){
    samples = unique(sample_data$sample_id[sample_data$classification %in% c('acs_subacute', 'acs_w_infection', 'acs_w_o_infection')])  # only acs comparison
    }
    

In [46]:
length(samples)

[1] 149

In [47]:
#samples

In [48]:
length(unique(data_long$sample_id))

[1] 146

In [49]:
data_long = data_long[data_long$sample_id %in% samples,]

In [50]:
length(unique(data_long$sample_id))

[1] 146

### Filter out variables

In [51]:
head(data_long,2)

,X,sample_id,type,variable,value,config
,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,1,k10,single_cell,X0_T.cell.CD4__SSU72,0.5896835,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-
2,2,k10,single_cell,X0_T.cell.CD4__PARK7,0.8587511,FALSE-FALSE-TRUE-0.2-FALSE-TRUE-


In [52]:
unique(data_long$type)

[1] "single_cell"   "cytokine"      "proteomics"    "neutrophil"   
[5] "clinical_data"

In [53]:
data_long = data_long[data_long$variable != 'delta_ef_value',]

In [54]:
unique(data_long[data_long$type == 'clinical_data', c('type', 'variable')])

,type,variable
,<chr>,<chr>
1557210,clinical_data,CK
1557356,clinical_data,CK_MB
1557502,clinical_data,Troponin
1557794,clinical_data,CRP


### Filter on expression

In [55]:
data_long$expression = TRUE

In [56]:
data_long$expression[data_long$value == 0] = FALSE

In [57]:
expression_filter = data_long %>% group_by(type, variable) %>% summarise(perc_expression = sum(expression)  )

`summarise()` has grouped output by 'type'. You can override using the `.groups` argument.


In [58]:
expression_filter$perc_expression = expression_filter$perc_expression / length(unique(data_long$sample_id))

In [59]:
nrow(expression_filter)

[1] 13288

In [60]:
nrow(unique(data_long[,c('type', 'variable')]))

[1] 13288

In [61]:
unique(expression_filter$type[expression_filter$perc_expression <= 0.2])

[1] "cytokine"

In [62]:
filtered_out = expression_filter[expression_filter$perc_expression <= 0.2,]

In [63]:
expression_filter = expression_filter[expression_filter$perc_expression > 0.2,]

In [64]:
#filtered_out

In [65]:
head(expression_filter,2)

type,variable,perc_expression
<chr>,<chr>,<dbl>
clinical_data,CK,1
clinical_data,CK_MB,1


In [66]:
nrow(expression_filter)

[1] 13282

In [67]:
head(expression_filter)

type,variable,perc_expression
<chr>,<chr>,<dbl>
clinical_data,CK,1.0000000
clinical_data,CK_MB,1.0000000
clinical_data,CRP,1.0000000
clinical_data,Troponin,1.0000000
cytokine,BCA1__CXCL13,0.8630137
cytokine,CTACK__CCL27,0.8630137


In [68]:
data_long = merge(data_long, expression_filter[,c('type', 'variable')], by.x = c('type', 'variable'), by.y = c('type', 'variable'))   # filter the data

In [69]:
unique(data_long$type)

[1] "clinical_data" "cytokine"      "neutrophil"    "proteomics"   
[5] "single_cell"

In [70]:
nrow(data_long)

[1] 1557037

In [72]:
length(unique(data_long$variable))

[1] 13282

In [73]:
length(unique(data_long$sample_id))

[1] 146

### Normalization & wide format

In [74]:
### Standardize values

In [75]:
standardize

[1] FALSE

In [76]:
if(standardize == TRUE){
    data_long = merge(data_long, data_long %>% group_by(variable) %>% summarise(mean = mean(value, na.rm = TRUE), sd = sd(value, na.rm = TRUE)))
    
    data_long[data_long == 0] = NA
    
    data_long = data_long[(data_long$sd != 0) & (!is.na(data_long$sd)),]
    
    data_nas = data.frame(is.na(data_long))
    data_long$value = (data_long$value - data_long$mean)/data_long$sd
    
    #data_long = data.frame(data_long)
    data_long$mean = NULL
    data_long$sd = NULL
    data_long$value[data_nas$value] = NA
    }

In [77]:
unique(data_long$type)

[1] "clinical_data" "cytokine"      "neutrophil"    "proteomics"   
[5] "single_cell"

In [78]:
## Prepare wide format for correlations

In [79]:
data_long$ident = paste0(data_long$type, '_0_', data_long$variable)

In [80]:
nrow(unique(data_long[,c('sample_id', 'ident')]))

[1] 1557037

In [81]:
nrow(data_long)

[1] 1557037

In [83]:
### Transform to wide

In [84]:
final_data = dcast(data_long, sample_id ~ ident , value.var = "value") # ! with this merging there might be NA values for some samples on some data types

Warning message in dcast(data_long, sample_id ~ ident, value.var = "value"):
“The dcast generic in data.table has been passed a data.frame and will attempt to redirect to the reshape2::dcast; please note that reshape2 is deprecated, and this redirection is now deprecated as well. Please do this redirection yourself like reshape2::dcast(data_long). In the next version, this warning will become an error.”


In [86]:
rownames(final_data) = final_data$sample_id

In [87]:
final_data$sample_id = NULL

In [89]:
ncol(final_data)

[1] 13282

In [90]:
nrow(final_data)

[1] 146

### Deal with NA - Set NA for 0 observation + remove samples with only NA

In [92]:
set_zero_na

[1] FALSE

In [93]:
if(set_zero_na == TRUE){
    final_data[final_data == 0] = NA
    }

In [94]:
### Remember NA's

In [95]:
data_nas = is.na(final_data)

In [96]:
head(data_nas,2)

,clinical_data_0_CK,clinical_data_0_CK_MB,clinical_data_0_CRP,clinical_data_0_Troponin,cytokine_0_BCA1__CXCL13,cytokine_0_CTACK__CCL27,cytokine_0_EGF__EGF,cytokine_0_ENA78__CXCL5,cytokine_0_Eotaxin__CCL11,cytokine_0_Eotaxin2__CCL24,⋯,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZEB2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAND5,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFAS1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L1,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZFP36L2,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF106,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF207,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZNF706,single_cell_0_X9_Monocytes...CD16_FCGR3A__ZYX
k1,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
k10,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,⋯,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [97]:
rowSums(data_nas)

k1   k10   k11   k12   k13   k14   k15   k16   k17   k18   k19    k2   k20 
12322     3     1     2     1     1   893     2     1   894     2   895     3 
  k21   k22   k23   k24   k25   k26   k27   k28   k29    k3   k30   k31   k32 
  893     0   893   893     0     0   893     2  1386     4     0   893     2 
  k33   k34    k4    k5    k6    k7    k8    k9  m1.1  m1.2  m1.3  m1.4 m10.1 
    1     2   893     0     1     1     0   896   892 13282 13282 13282 11831 
m10.2 m10.3 m10.4 m11.1 m11.2 m11.3 m11.4 m12.1 m12.2 m12.3 m12.4 m13.1 m13.2 
    1     0 13282 12723     0     1     2   893     0 13282 13282     0   892 
m13.3 m13.4 m14.1 m14.2 m14.3 m14.4 m15.1 m15.2 m15.3 m15.4 m16.1 m16.2 m16.3 
    0     1     0   892 11831 11832     0   892     0     0   892     0 13282 
m16.4 m17.1 m17.2 m17.3 m17.4 m18.1 m18.2 m18.3 m18.4 m19.1 m19.2 m19.3 m19.4 
13282     0 13282 13282 13282     0   892     1 13282     0     0   893 13282 
 m2.1  m2.2  m2.3  m2.4 m20.1 m20.2 m20.3 m20.4 m21.1 m21.2 m21.3 m21.4 m22.1 
    1   892   892   893 12724 11831   892     1     0     0     2     2     0 
m22.2 m22.3 m22.4 m23.1 m23.2 m23.3 m23.4 m24.1 m24.2 m24.3 m24.4 m25.1 m25.2 
    0   893     0     1   490  1447     1 12321     0     2     3     2     1 
m25.3 m25.4 m26.1 m26.2 m26.3 m26.4 m27.1 m27.2 m27.3 m27.4 m28.1 m28.2 m28.3 
    2 13282   490     0   892   556     1     0     0 13282 13282     0     1 
m28.4  m3.1  m3.2  m3.3  m3.4  m4.1  m4.2  m4.3  m4.4  m5.1  m5.2  m5.3  m5.4 
    2 13282     0     0     0   490     1     0     2     0 12723     0   893 
 m6.1  m6.2  m6.3  m6.4  m7.1  m7.2  m7.3  m7.4  m8.1  m8.2  m8.3  m8.4  m9.1 
 1382   893     0     1   892     1     0   894     1     1     2     3   892 
 m9.2  m9.3  m9.4 
  893     0 13282

In [98]:
ncol(final_data)

[1] 13282

In [99]:
keep_samples = names(rowSums(data_nas))[rowSums(data_nas) != ncol(final_data)]

In [100]:
final_data = final_data[keep_samples,]

In [101]:
data_nas = data_nas[keep_samples,]

In [102]:
nrow(final_data)

[1] 128

### Apply feature wise quantile normalization

In [108]:
quantile_norm_feat

[1] TRUE

In [109]:
if(quantile_norm_feat == TRUE){
    final_data = apply(final_data, 2,stdnorm)
    final_data = data.frame(final_data)
    final_data[data_nas] = NA
    final_data$sample_id = rownames(final_data)
    data_long = melt(final_data)
    data_long$type = str_extract(data_long$variable, '.*_0_')
    data_long$type  = str_replace(data_long$type , '_0_', '')
    data_long$variable = str_replace(data_long$variable, '.*_0_', '')
    }

Warning message in melt(final_data):
“The melt generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is deprecated, and this redirection is now deprecated as well. To continue using melt methods from reshape2 while both libraries are attached, e.g. melt.list, you can prepend the namespace like reshape2::melt(final_data). In the next version, this warning will become an error.”
Using sample_id as id variables



In [111]:
length(unique(data_long$variable))

[1] 13282

# Save Prepared Data

In [113]:
name_save

[1] "V_FINAL_INTEGRATED"

In [115]:
write.csv(data_long, paste0(result_path, '/E-Analysis/Combined_Data_', name_save, '.csv'))

In [116]:
length(unique(data_long$variable))

[1] 13282